<a href="https://colab.research.google.com/github/StevenGarc/Ejercicios_IA/blob/master/DT_entropia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import math

In [ ]:
age = [1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3]
spec= [1,1,1,1,2,2,2,2,1,1,1,1,2,2,2,2,1,1,1,1,2,2,2,2]
astig=[1,1,2,2,1,1,2,2,1,1,2,2,1,1,2,2,1,1,2,2,1,1,2,2]
tears=[1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2]
Class=[3,2,3,1,3,2,3,1,3,2,3,1,3,2,3,3,3,3,3,1,3,2,3,3]
data = {'age': age, 
        'specRx':spec, 
        'asig':astig, 
        'tears':tears, 
        'Class':Class}
df = pd.DataFrame(data)
df

,age,specRx,asig,tears,Class
0,1,1,1,1,3
1,1,1,1,2,2
2,1,1,2,1,3
3,1,1,2,2,1
4,1,2,1,1,3
5,1,2,1,2,2
6,1,2,2,1,3
7,1,2,2,2,1
8,2,1,1,1,3
9,2,1,1,2,2


In [ ]:
def cal_entropia(df, clases):
  
  conteo = {}
  total = 0
  entropia_inicial = 0
  entropias = {}
  for index, row in df.iterrows():
    c = row[clases]
    conteo[c] = 1 if c not in conteo else conteo[c]+1
    total+=1

  for k, v in conteo.items():
    p=v/total
    entropia_inicial -= p* math.log(p,2)
  
  return entropia_inicial

In [ ]:
def dividir(df, atributo):
  sub_dataset = {}
  aux_df = df[atributo]
  aux_df = aux_df.drop_duplicates()
  for i in aux_df:
    col_i = df[df[atributo]==i]
    sub_dataset[i]=col_i
  return sub_dataset


In [ ]:
def gain (df, clases, entropia_inicial):
  
  entropias = {}
  total = len(df)

  for columna in df.columns:
    if columna != clases:
      aux_df = dividir(df, columna)
      entropias[columna] = 0 
      for k, i in aux_df.items():
        E_i = cal_entropia(i, clases)
        entropias[columna] += (len(i)/total) * E_i

  gain = 0
  atributo = clases

  for k, v in entropias.items():
    new_en = entropia_inicial - v
    if gain <= new_en: 
      atributo = k
      gain = new_en

  return gain, atributo


gain(df, 'Class', cal_entropia(df, 'Class'))

(0.5487949406953986, 'tears')

In [ ]:
def tree(df, clases):
  Tree = {}
  entropia_inicial = cal_entropia(df, clases)
  ganancia, atributo = gain(df, clases, entropia_inicial)
  
  Tree[atributo] = {}
  for k, v in dividir(df, atributo).items():
    Tree[atributo][k] = tree(v.drop([atributo], axis=1), clases) if cal_entropia(v, clases) != 0 else v['Class'].tolist()[0]

  return Tree
  

In [ ]:
def evaluar(row, t):
  while type(t)==dict:
    n = list(t)[0] #atributo
    t = t[n]
    num = row[n] #numero
    t = t[num]
  return t
      

In [ ]:
t = tree(df, 'Class')
t

{'tears': {1: 3,
  2: {'asig': {1: {'age': {1: 2, 2: 2, 3: {'specRx': {1: 3, 2: 2}}}},
    2: {'specRx': {1: 1, 2: {'age': {1: 1, 2: 3, 3: 3}}}}}}}}

In [ ]:
for index, row in df.iterrows():
  x = evaluar(row, t)
  print(x, row['Class'])

3 3
2 2
3 3
1 1
3 3
2 2
3 3
1 1
3 3
2 2
3 3
1 1
3 3
2 2
3 3
3 3
3 3
3 3
3 3
1 1
3 3
2 2
3 3
3 3
